### Import necessary libraries

In [ ]:
import numpy as np
from scipy.optimize import minimize_scalar
from plot_descent_penalty import plot_2d_contour, plot_3d_surface

In [ ]:
# ACHAR OUTRO PROBLEMA
# MODIFICAR ALGORITMOS DOS PLOTS
# ORGANIZAR MELHOR O CÓDIGO
# SQP

### Problems to optimize (define a new problem here)

In [ ]:
def get_problem(problem):
    """Get the problem to optimize.
    Problem 1: Example 5.8 (Joaquim R. R. A. Martins, Andrew Ning - Engineering Design Optimization (2021))
    Problem 2: Something from nothing

    Args:
        problem (int): problem selected

    Returns:
        obj_fun (dict): contains the objective function (key = 'fun')
                        and its gradient (key = 'jac')
        eq_cons (dict): contains the equality constraints (key = 'fun')
                        and its gradients (key = 'jac')
        ineq_cons (dict): contains the inequality constraints (key = 'fun')
                          and its gradients (key = 'jac')
    """
    
    match problem:
        # Problem 1 - Example 5.8 (Joaquim R. R. A. Martins, Andrew Ning - Engineering Design Optimization (2021))
        case 1:
            # Objective function
            obj_fun = {
                'type': 'obj',
                'fun' : lambda x : np.array([x[0] + 2*x[1]]),
                'jac' : lambda x : np.array([1, 2])
            }
            
            # Equality constraints
            eq_cons = {
                'type': 'eq',
                'fun' : lambda x: np.array([]),
                'jac' : lambda x: np.array([])
                }
            
            # Inequality constraints
            ineq_cons = {
                'type': 'ineq',
                'fun' : lambda x: np.array([1/4*x[0]**2 + x[1]**2 - 1]),
                'jac' : lambda x: np.array([1/2*x[0], 2*x[1]])
                }
        
        # Problem 2 - Something from nothing
        case 2:
            # Objective function
            obj_fun = {
                'type': 'obj',
                'fun' : lambda x : np.array([x[0] + 2*x[1]]),
                'jac' : lambda x : np.array([1, 2])
            }
            
            # Equality constraints
            eq_cons = {
                'type': 'eq',
                'fun' : lambda x: np.array([]),
                'jac' : lambda x: np.array([])
                }
            
            # Inequality constraints
            ineq_cons = {
                'type': 'ineq',
                'fun' : lambda x: np.array([1/4*x[0]**2 + x[1]**2 - 1]),
                'jac' : lambda x: np.array([1/2*x[0], 2*x[1]])
                }
    
    return obj_fun, eq_cons, ineq_cons

### Exterior penalty methods

In [ ]:
def get_method(method):
    """Definition of the penalized Lagrangian (transformed functional phi)

    Args:
        method (int): selected function phi

    Returns:
        phi (function): a method that receives a point 'x' (numpy array of shape (2,))
                        and returns the value of phi and its gradient
    """
    
    match method:
        # Quadratic penalty
        case 1:
            def phi(x):
                f, df, h, dh, g, dg = get_values(x)
                
                auxg = np.maximum(0, g)
                ph = f + mu*(h.sum()**2 + auxg.sum()**2)
                
                # Construction of the gradient of phi: contribution of the inequality constraints (g_i)
                dgaux = np.array(np.zeros(x.shape))
                if g.size == 1:  # split into two situations: with only one constraints, and more constraints
                    dgaux = dg*np.maximum(0, g)
                else:
                    for i in range(g.size):
                        dgaux = dgaux + dg[i, :]*np.maximum(0, g[i])
                    
                # Construction of the gradient of phi: contribution of the equality constraints (h_j)
                dhaux = np.array(np.zeros(x.shape))
                if h.size == 1:  # split into two situations: with only one constraints, and more constraints
                    dhaux = dh*h
                else:
                    for j in range(h.size):
                        dhaux = dhaux + dh[j, :]*h[j]
                
                # Gradient of phi:
                dph = df + 2*mu*(dhaux + dgaux)
                
                return ph, dph
        
        # Augmented Lagrangian
        case 2:
            def phi(x):
                f, df, h, dh, g, dg = get_values(x)
                
                ghat = np.copy(g)
                for i in range(g.size):
                    c = -lambda_ineq[i]/mu
                    if g[i] < c:
                        ghat[i] = c

                # maxg = np.max(0, ghat)
                L = f + np.dot(lambda_eq, h) + np.dot(lambda_ineq, ghat) + mu/2 * (np.dot(h, h) + np.dot(ghat, ghat))
                
                # transform in matrix
                if dh.size > 0 and len(dh.shape) == 1:
                    dh = np.array([dh])
                    
                if dg.size > 0 and len(dg.shape) == 1:
                    dg = np.array([dg])
                    
                dL = df + np.matmul(lambda_eq, dh) + np.matmul(lambda_ineq, dg) + mu*(np.matmul(h, dh) + np.matmul(g, dg))
                
                return L, dL
    
    return phi

In [ ]:
def get_values(x):
    """Get the values of the objective function,
    equality constraints, inequality constraints
    and all its gradients

    Args:
        x (numpy array): point [x1, x2]

    Returns:
        values of gradients
    """
    obj_fun, eq_cons, ineq_cons = get_problem(problem)
    
    f, df = obj_fun['fun'](x), obj_fun['jac'](x)
    h, dh = eq_cons['fun'](x), eq_cons['jac'](x)
    g, dg = ineq_cons['fun'](x), ineq_cons['jac'](x)
    
    return f, df, h, dh, g, dg

# Set parameters (user has to define)

#### Problem to run

In [ ]:
# 1 = Example 5.8 (Joaquim R. R. A. Martins, Andrew Ning - Engineering Design Optimization (2021))
# 2 = Something from nothing
problem = 1

#### Initial guess

In [ ]:
x0 = 5*np.array([1, 1])

#### Upper bound for the Golden Search Algorithm

In [ ]:
alpha0 = 1.0

#### Initial value for penalization (mu) and rate of increase (rho)

In [ ]:
mu = 0.1
rho = 2

#### Convergence tolerance for the minimization of Lagrangian

In [ ]:
TolG = 1e-5

#### Stopping criterias for the exterior penalty method

In [ ]:
itmax = 10 # Maximum number of iterations
epsilon1 = 0.001 # Magnitude of the penalty terms
epsilon2 = 0.001 # Change in value of the penalized objective function

#### Define the method of optimization

In [ ]:
# 1 = Quadratic penalty
# 2 = Augmented Lagrangian
method = 1

## Get the problem and method

In [ ]:
objective_function, equality_constraints, inequality_constraints = get_problem(problem)
phi = get_method(method)

## Optimize

### Conjugate Gradient + Golden Search

In [ ]:
def f_alpha(alpha, args):
    """Definition of the equation to be minimized as function of the step size alpha

    Args:
        alpha (float): step size
        args (array): array with arguments point (x) and direction of search (d)

    Returns:
        f (float): value of phi
    """
    x = args[0] + alpha*args[1]
    f, _ = phi(x)
    
    return f

In [ ]:
def CG_GS(x, alpha0, TolG):
    # Count variable
    t = 0
    # f and df values at the initial point
    [f, df] = phi(x)
    dftm1 = df
    
    xs = [x]
    fs = [f]
        
    while np.sqrt(df @ df) > TolG:
        # Search direction: Conjugated Gradient
        beta = (np.linalg.norm(df)/np.linalg.norm(dftm1))**2
 
        if t == 0:
            d = -df
        else:
            d = -df + beta*dtm1
            
        # Step determination: Golden Search (method='golden'), Brent (method='brent') or Bounded (method='bounded')
        alpha = minimize_scalar(f_alpha, bounds=(.001, alpha0), args=([x, d]), method='bounded')

        # Update the current point
        xt = x + alpha.x*d
        xs.append(xt)
        
        # Saves information of gradient and descent direction of current iteration
        dftm1 = df
        dtm1 = d
    
        # Evaluate the objective function and gradient at the new point
        [f, df] = phi(xt)
        
        fs.append(f)
    
        # Update the design variable and iteration number
        x = xt
        t = t + 1
    
    return x, f, df, t, xs, fs

### Exterior penalty

In [ ]:
k, stop_1, stop_2 = 0, 1.0, 1.0 # for stopping criterias
cost_f, cost_g = 0, 0 # costs

_, _, h, _, g, _ = get_values(np.array([0, 0]))

lambda_eq = np.zeros((h.size))
lambda_ineq = np.zeros((g.size))

points = []
values = []

while k < itmax and stop_1 > epsilon1 and stop_2 > epsilon2:
    # Conjugate gradient + Golden search method
    xt, f, df, t, xs, fs = CG_GS(x, alpha0, TolG) # minimize
    
    # Check convergence
    fopt, _, h, dh, g, dg = get_values(xt)
    stop_1 = abs((f - fopt)/fopt)
    
    if k > 0:
        stop_2 = abs((f - f_old)/f)
        
    points.append(xs)
    values.append(fs)
    
    f_old = f
    
    if k >= itmax:
        print('Stopped due to the number of iterations')
    elif stop_1 <= epsilon1:
        print('Stopped due to the small magnitude of the penalty terms')
    elif stop_2 <= epsilon2:
        print('Stopped due to a small change in value of the penalized objective function')
        
    # Update Lagrange multiplier, penalty parameter and starting point
    lambda_eq = lambda_eq + mu*h
    lambda_ineq = lambda_ineq + mu*g
    mu = mu*rho
    x = xt
    k = k + 1
    
    # Update cost
    cost_f += t
    cost_g += t

## Print results

In [ ]:
fopt, dfopt, hopt, dhopt, gopt, dgopt = get_values(x)

print('Optimum found:')
print(xt)
print('Objective function value at the optimum:')
print(fopt)
print('Inequality constraints at the optimum:')
print(gopt)
print('Equality constraints at the optimum:')
print(hopt)

print('Number of times that the f_obj function and constraints were evaluated, respectively:')
print(cost_f)
print(cost_g)
print('Number of iterations of the External penalty method:')
print(k)

## Plot

In [ ]:
all_x = np.array(points)
all_x = np.reshape(all_x, ( int(all_x.size/2), 2 ))

all_f = np.array(values)
all_f = np.reshape(all_f, (all_f.size))

plot_2d_contour(all_x, obj_fun=objective_function, plot_h=True, plot_g=True, eq_constraints=equality_constraints, ineq_constraints=inequality_constraints)
plot_3d_surface(all_x, all_f, obj_fun=objective_function, plot_h=True, plot_g=True, eq_constraints=equality_constraints, ineq_constraints=inequality_constraints)